# Week 17 - Day 2: Implied Volatility

## Learning Objectives
- Understand implied volatility (IV) and its importance in options pricing
- Implement IV calculation using numerical methods (Newton-Raphson, Bisection)
- Build and visualize volatility surfaces
- Analyze term structure of volatility
- Understand volatility smile and skew patterns

## Topics Covered
1. Black-Scholes Model Review
2. Implied Volatility Calculation Methods
3. Volatility Surface Construction
4. Term Structure Analysis
5. Practical Applications

In [ ]:
# Import required libraries
import numpy as np
from scipy import optimize
from scipy.stats import norm
from scipy.interpolate import griddata, RectBivariateSpline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

---
## 1. Black-Scholes Model Review

The Black-Scholes formula for European options:

**Call Option:**
$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$

**Put Option:**
$$P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

**Implied Volatility** is the volatility $\sigma$ that, when input into Black-Scholes, produces the observed market price.

In [ ]:
class BlackScholes:
    """
    Black-Scholes option pricing model implementation.
    """
    
    @staticmethod
    def d1(S, K, T, r, sigma):
        """Calculate d1 parameter."""
        return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    
    @staticmethod
    def d2(S, K, T, r, sigma):
        """Calculate d2 parameter."""
        return BlackScholes.d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    @staticmethod
    def call_price(S, K, T, r, sigma):
        """
        Calculate European call option price.
        
        Parameters:
        -----------
        S : float - Current stock price
        K : float - Strike price
        T : float - Time to maturity (in years)
        r : float - Risk-free interest rate
        sigma : float - Volatility
        
        Returns:
        --------
        float - Call option price
        """
        if T <= 0:
            return max(S - K, 0)
        
        d1 = BlackScholes.d1(S, K, T, r, sigma)
        d2 = BlackScholes.d2(S, K, T, r, sigma)
        
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    @staticmethod
    def put_price(S, K, T, r, sigma):
        """
        Calculate European put option price.
        """
        if T <= 0:
            return max(K - S, 0)
        
        d1 = BlackScholes.d1(S, K, T, r, sigma)
        d2 = BlackScholes.d2(S, K, T, r, sigma)
        
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    @staticmethod
    def vega(S, K, T, r, sigma):
        """
        Calculate option vega (sensitivity to volatility).
        Vega is the same for calls and puts.
        """
        if T <= 0:
            return 0
        
        d1 = BlackScholes.d1(S, K, T, r, sigma)
        return S * norm.pdf(d1) * np.sqrt(T)


# Test the implementation
bs = BlackScholes()

# Example parameters
S = 100      # Stock price
K = 100      # Strike price
T = 0.25     # 3 months
r = 0.05     # 5% risk-free rate
sigma = 0.20 # 20% volatility

call = bs.call_price(S, K, T, r, sigma)
put = bs.put_price(S, K, T, r, sigma)
vega = bs.vega(S, K, T, r, sigma)

print("Black-Scholes Pricing Example")
print("=" * 40)
print(f"Stock Price (S):     ${S:.2f}")
print(f"Strike Price (K):    ${K:.2f}")
print(f"Time to Maturity:    {T:.2f} years")
print(f"Risk-free Rate:      {r:.2%}")
print(f"Volatility:          {sigma:.2%}")
print("-" * 40)
print(f"Call Price:          ${call:.4f}")
print(f"Put Price:           ${put:.4f}")
print(f"Vega:                {vega:.4f}")

# Verify put-call parity: C - P = S - K*exp(-rT)
parity_lhs = call - put
parity_rhs = S - K * np.exp(-r * T)
print(f"\nPut-Call Parity Check:")
print(f"C - P = {parity_lhs:.4f}")
print(f"S - Ke^(-rT) = {parity_rhs:.4f}")

---
## 2. Implied Volatility Calculation

Implied volatility cannot be solved analytically - we need numerical methods:

1. **Newton-Raphson Method**: Fast convergence using Vega
2. **Bisection Method**: Robust but slower
3. **Brent's Method**: Combines benefits of both

### Newton-Raphson Method
$$\sigma_{n+1} = \sigma_n - \frac{C_{BS}(\sigma_n) - C_{market}}{\text{Vega}(\sigma_n)}$$

In [ ]:
class ImpliedVolatility:
    """
    Implied Volatility calculator using multiple numerical methods.
    """
    
    @staticmethod
    def newton_raphson(market_price, S, K, T, r, option_type='call',
                       initial_guess=0.2, tol=1e-8, max_iter=100):
        """
        Calculate IV using Newton-Raphson method.
        
        Parameters:
        -----------
        market_price : float - Observed market price of the option
        S, K, T, r : float - Option parameters
        option_type : str - 'call' or 'put'
        initial_guess : float - Starting volatility estimate
        tol : float - Convergence tolerance
        max_iter : int - Maximum iterations
        
        Returns:
        --------
        float - Implied volatility
        """
        sigma = initial_guess
        
        for i in range(max_iter):
            # Calculate theoretical price
            if option_type == 'call':
                price = BlackScholes.call_price(S, K, T, r, sigma)
            else:
                price = BlackScholes.put_price(S, K, T, r, sigma)
            
            # Calculate vega
            vega = BlackScholes.vega(S, K, T, r, sigma)
            
            # Price difference
            diff = price - market_price
            
            # Check convergence
            if abs(diff) < tol:
                return sigma
            
            # Avoid division by zero
            if abs(vega) < 1e-10:
                return np.nan
            
            # Newton-Raphson update
            sigma = sigma - diff / vega
            
            # Keep sigma positive
            sigma = max(sigma, 1e-6)
        
        return np.nan  # Did not converge
    
    @staticmethod
    def bisection(market_price, S, K, T, r, option_type='call',
                  sigma_low=0.001, sigma_high=3.0, tol=1e-8, max_iter=100):
        """
        Calculate IV using Bisection method.
        More robust than Newton-Raphson but slower.
        """
        price_func = BlackScholes.call_price if option_type == 'call' else BlackScholes.put_price
        
        for i in range(max_iter):
            sigma_mid = (sigma_low + sigma_high) / 2
            price_mid = price_func(S, K, T, r, sigma_mid)
            
            if abs(price_mid - market_price) < tol:
                return sigma_mid
            
            if price_mid > market_price:
                sigma_high = sigma_mid
            else:
                sigma_low = sigma_mid
        
        return sigma_mid
    
    @staticmethod
    def brent(market_price, S, K, T, r, option_type='call',
              sigma_low=0.001, sigma_high=3.0):
        """
        Calculate IV using scipy's Brent method.
        Combines reliability of bisection with speed of secant method.
        """
        price_func = BlackScholes.call_price if option_type == 'call' else BlackScholes.put_price
        
        def objective(sigma):
            return price_func(S, K, T, r, sigma) - market_price
        
        try:
            result = optimize.brentq(objective, sigma_low, sigma_high)
            return result
        except ValueError:
            return np.nan


# Test IV calculation
iv_calc = ImpliedVolatility()

# Generate a market price using known volatility
true_sigma = 0.25
market_call_price = BlackScholes.call_price(S, K, T, r, true_sigma)

print(f"True Volatility: {true_sigma:.4f}")
print(f"Market Call Price: ${market_call_price:.4f}")
print("\nImplied Volatility Calculations:")
print("-" * 40)

# Newton-Raphson
iv_nr = iv_calc.newton_raphson(market_call_price, S, K, T, r)
print(f"Newton-Raphson IV:  {iv_nr:.6f}")

# Bisection
iv_bisect = iv_calc.bisection(market_call_price, S, K, T, r)
print(f"Bisection IV:       {iv_bisect:.6f}")

# Brent
iv_brent = iv_calc.brent(market_call_price, S, K, T, r)
print(f"Brent IV:           {iv_brent:.6f}")

In [ ]:
# Convergence comparison
def track_convergence_nr(market_price, S, K, T, r, initial_guess=0.5, max_iter=20):
    """Track Newton-Raphson convergence."""
    sigma = initial_guess
    history = [sigma]
    
    for i in range(max_iter):
        price = BlackScholes.call_price(S, K, T, r, sigma)
        vega = BlackScholes.vega(S, K, T, r, sigma)
        
        if abs(vega) < 1e-10:
            break
            
        sigma = sigma - (price - market_price) / vega
        sigma = max(sigma, 1e-6)
        history.append(sigma)
        
        if abs(history[-1] - history[-2]) < 1e-10:
            break
    
    return history


def track_convergence_bisect(market_price, S, K, T, r, sigma_low=0.01, sigma_high=1.0, max_iter=50):
    """Track Bisection convergence."""
    history = []
    
    for i in range(max_iter):
        sigma_mid = (sigma_low + sigma_high) / 2
        history.append(sigma_mid)
        
        price_mid = BlackScholes.call_price(S, K, T, r, sigma_mid)
        
        if abs(price_mid - market_price) < 1e-10:
            break
            
        if price_mid > market_price:
            sigma_high = sigma_mid
        else:
            sigma_low = sigma_mid
    
    return history


# Compare convergence
history_nr = track_convergence_nr(market_call_price, S, K, T, r, initial_guess=0.5)
history_bisect = track_convergence_bisect(market_call_price, S, K, T, r)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Convergence plot
axes[0].plot(history_nr, 'b-o', label='Newton-Raphson', markersize=8)
axes[0].plot(history_bisect[:15], 'r-s', label='Bisection', markersize=6)
axes[0].axhline(y=true_sigma, color='green', linestyle='--', label=f'True σ = {true_sigma}')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Implied Volatility')
axes[0].set_title('Convergence Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error plot (log scale)
errors_nr = np.abs(np.array(history_nr) - true_sigma)
errors_bisect = np.abs(np.array(history_bisect[:15]) - true_sigma)

axes[1].semilogy(errors_nr, 'b-o', label='Newton-Raphson', markersize=8)
axes[1].semilogy(errors_bisect, 'r-s', label='Bisection', markersize=6)
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Absolute Error (log scale)')
axes[1].set_title('Convergence Rate')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nNewton-Raphson converged in {len(history_nr)} iterations")
print(f"Bisection converged in {len(history_bisect)} iterations")

---
## 3. Volatility Smile and Skew

In reality, implied volatility varies with strike price, creating patterns known as:

- **Volatility Smile**: IV higher for OTM puts and calls (symmetric)
- **Volatility Skew**: IV higher for OTM puts than OTM calls (asymmetric)

These patterns arise because:
1. Black-Scholes assumes constant volatility (unrealistic)
2. Market fears crashes more than rallies
3. Supply/demand for options at different strikes

In [ ]:
def generate_realistic_iv_smile(strikes, S, atm_iv=0.20, skew=-0.1, smile=0.02):
    """
    Generate realistic implied volatility smile/skew pattern.
    
    Parameters:
    -----------
    strikes : array - Strike prices
    S : float - Current stock price
    atm_iv : float - At-the-money implied volatility
    skew : float - Skew parameter (negative = put skew)
    smile : float - Smile curvature parameter
    
    Returns:
    --------
    array - Implied volatilities for each strike
    """
    # Moneyness (log scale)
    moneyness = np.log(strikes / S)
    
    # SABR-like smile pattern
    iv = atm_iv + skew * moneyness + smile * moneyness**2
    
    return np.maximum(iv, 0.01)  # Ensure positive IV


# Generate strikes around ATM
S = 100
strikes = np.linspace(70, 130, 25)
T = 0.25
r = 0.05

# Generate IVs with different patterns
iv_smile = generate_realistic_iv_smile(strikes, S, atm_iv=0.20, skew=0, smile=0.015)
iv_skew = generate_realistic_iv_smile(strikes, S, atm_iv=0.20, skew=-0.15, smile=0.01)
iv_flat = np.ones_like(strikes) * 0.20

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IV vs Strike
axes[0].plot(strikes, iv_flat * 100, 'k--', label='Flat (BS assumption)', linewidth=2)
axes[0].plot(strikes, iv_smile * 100, 'b-o', label='Smile Pattern', linewidth=2, markersize=4)
axes[0].plot(strikes, iv_skew * 100, 'r-s', label='Skew Pattern', linewidth=2, markersize=4)
axes[0].axvline(x=S, color='gray', linestyle=':', alpha=0.7, label='ATM')
axes[0].set_xlabel('Strike Price')
axes[0].set_ylabel('Implied Volatility (%)')
axes[0].set_title('Volatility Smile vs Skew')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# IV vs Moneyness
moneyness = strikes / S
axes[1].plot(moneyness, iv_flat * 100, 'k--', label='Flat', linewidth=2)
axes[1].plot(moneyness, iv_smile * 100, 'b-o', label='Smile', linewidth=2, markersize=4)
axes[1].plot(moneyness, iv_skew * 100, 'r-s', label='Skew', linewidth=2, markersize=4)
axes[1].axvline(x=1.0, color='gray', linestyle=':', alpha=0.7)
axes[1].set_xlabel('Moneyness (K/S)')
axes[1].set_ylabel('Implied Volatility (%)')
axes[1].set_title('Volatility by Moneyness')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Generate synthetic option prices and recover IV
def generate_option_data_with_iv(S, strikes, T, r, true_ivs):
    """
    Generate option prices from known IVs and then recover IV.
    """
    results = []
    
    for K, true_iv in zip(strikes, true_ivs):
        # Generate call price with known IV
        call_price = BlackScholes.call_price(S, K, T, r, true_iv)
        put_price = BlackScholes.put_price(S, K, T, r, true_iv)
        
        # Add small noise to simulate market prices
        noise = np.random.normal(0, 0.01 * call_price)
        market_call = max(call_price + noise, 0.01)
        
        # Recover IV
        recovered_iv = ImpliedVolatility.brent(market_call, S, K, T, r, 'call')
        
        results.append({
            'strike': K,
            'moneyness': K / S,
            'call_price': market_call,
            'put_price': put_price,
            'true_iv': true_iv,
            'recovered_iv': recovered_iv
        })
    
    return results


# Generate data
option_data = generate_option_data_with_iv(S, strikes, T, r, iv_skew)

# Display sample
print("Option Data with Implied Volatility")
print("=" * 80)
print(f"{'Strike':<10} {'K/S':<10} {'Call Price':<12} {'True IV':<12} {'Recovered IV':<12} {'Error'}")
print("-" * 80)

for d in option_data[::3]:  # Every 3rd row
    error = abs(d['recovered_iv'] - d['true_iv']) * 10000  # in bps
    print(f"{d['strike']:<10.2f} {d['moneyness']:<10.2f} ${d['call_price']:<11.4f} "
          f"{d['true_iv']*100:<11.2f}% {d['recovered_iv']*100:<11.2f}% {error:.2f} bps")

---
## 4. Volatility Surface Construction

A **Volatility Surface** maps implied volatility across two dimensions:
- **Strike (or Moneyness)**: X-axis
- **Time to Expiration**: Y-axis

This provides a complete picture of market volatility expectations.

In [ ]:
class VolatilitySurface:
    """
    Build and analyze implied volatility surfaces.
    """
    
    def __init__(self, S, r):
        """
        Initialize volatility surface.
        
        Parameters:
        -----------
        S : float - Current stock price
        r : float - Risk-free rate
        """
        self.S = S
        self.r = r
        self.strikes = None
        self.expirations = None
        self.iv_matrix = None
    
    def generate_surface(self, strikes, expirations, 
                         base_iv=0.20, skew=-0.12, smile=0.008,
                         term_slope=-0.05):
        """
        Generate a realistic volatility surface.
        
        Parameters:
        -----------
        strikes : array - Strike prices
        expirations : array - Times to expiration (in years)
        base_iv : float - Base ATM implied volatility
        skew : float - Skew parameter
        smile : float - Smile curvature
        term_slope : float - Term structure slope
        """
        self.strikes = strikes
        self.expirations = expirations
        
        # Create meshgrid
        K_grid, T_grid = np.meshgrid(strikes, expirations)
        
        # Moneyness
        moneyness = np.log(K_grid / self.S)
        
        # Generate IV surface
        # - Smile/skew in strike dimension
        # - Term structure in time dimension (shorter maturities often have higher IV)
        # - Smile flattens for longer maturities
        
        smile_component = skew * moneyness + smile * moneyness**2
        term_component = term_slope * np.log(T_grid / 0.25)  # Relative to 3-month
        
        # Smile dampening for longer maturities
        dampening = np.exp(-0.5 * T_grid)
        
        self.iv_matrix = base_iv + smile_component * dampening + term_component
        self.iv_matrix = np.maximum(self.iv_matrix, 0.05)  # Floor at 5%
        
        return self.iv_matrix
    
    def get_iv(self, K, T):
        """
        Interpolate IV for given strike and expiration.
        """
        if self.iv_matrix is None:
            raise ValueError("Surface not generated yet")
        
        # Create interpolator
        spline = RectBivariateSpline(self.expirations, self.strikes, self.iv_matrix)
        return float(spline(T, K))
    
    def plot_surface(self, title="Implied Volatility Surface"):
        """
        Create 3D visualization of the volatility surface.
        """
        K_grid, T_grid = np.meshgrid(self.strikes, self.expirations)
        
        fig = plt.figure(figsize=(14, 10))
        
        # 3D Surface
        ax1 = fig.add_subplot(2, 2, 1, projection='3d')
        surf = ax1.plot_surface(K_grid, T_grid * 365, self.iv_matrix * 100,
                                cmap='viridis', alpha=0.8, edgecolor='none')
        ax1.set_xlabel('Strike')
        ax1.set_ylabel('Days to Expiry')
        ax1.set_zlabel('IV (%)')
        ax1.set_title('3D Volatility Surface')
        ax1.view_init(elev=25, azim=45)
        
        # Contour plot
        ax2 = fig.add_subplot(2, 2, 2)
        contour = ax2.contourf(K_grid, T_grid * 365, self.iv_matrix * 100,
                               levels=20, cmap='viridis')
        ax2.axvline(x=self.S, color='red', linestyle='--', alpha=0.7, label='ATM')
        plt.colorbar(contour, ax=ax2, label='IV (%)')
        ax2.set_xlabel('Strike')
        ax2.set_ylabel('Days to Expiry')
        ax2.set_title('Volatility Surface Contour')
        ax2.legend()
        
        # Smile at different expirations
        ax3 = fig.add_subplot(2, 2, 3)
        for i, T in enumerate(self.expirations[::2]):
            idx = np.where(self.expirations == T)[0][0]
            ax3.plot(self.strikes, self.iv_matrix[idx, :] * 100,
                    label=f'{int(T*365)} days', linewidth=2)
        ax3.axvline(x=self.S, color='gray', linestyle=':', alpha=0.7)
        ax3.set_xlabel('Strike')
        ax3.set_ylabel('IV (%)')
        ax3.set_title('Volatility Smile by Expiration')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # Term structure at different strikes
        ax4 = fig.add_subplot(2, 2, 4)
        atm_idx = np.argmin(np.abs(self.strikes - self.S))
        otm_put_idx = np.argmin(np.abs(self.strikes - 0.9 * self.S))
        otm_call_idx = np.argmin(np.abs(self.strikes - 1.1 * self.S))
        
        ax4.plot(self.expirations * 365, self.iv_matrix[:, atm_idx] * 100,
                'b-o', label=f'ATM (K={self.strikes[atm_idx]:.0f})', linewidth=2)
        ax4.plot(self.expirations * 365, self.iv_matrix[:, otm_put_idx] * 100,
                'r-s', label=f'OTM Put (K={self.strikes[otm_put_idx]:.0f})', linewidth=2)
        ax4.plot(self.expirations * 365, self.iv_matrix[:, otm_call_idx] * 100,
                'g-^', label=f'OTM Call (K={self.strikes[otm_call_idx]:.0f})', linewidth=2)
        ax4.set_xlabel('Days to Expiry')
        ax4.set_ylabel('IV (%)')
        ax4.set_title('Term Structure by Strike')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        plt.suptitle(title, fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()


# Create volatility surface
S = 100
r = 0.05

strikes = np.linspace(70, 130, 25)
expirations = np.array([7, 14, 30, 60, 90, 120, 180, 270, 365]) / 365  # Convert to years

vol_surface = VolatilitySurface(S, r)
iv_matrix = vol_surface.generate_surface(strikes, expirations)

# Plot
vol_surface.plot_surface("SPY-like Implied Volatility Surface")

---
## 5. Term Structure of Volatility

The **term structure** shows how IV varies with time to expiration:

- **Normal/Contango**: Near-term IV < Long-term IV (calm markets)
- **Inverted/Backwardation**: Near-term IV > Long-term IV (stressed markets)

Term structure is crucial for:
- Calendar spread trading
- Volatility forecasting
- Risk management

In [ ]:
class TermStructureAnalysis:
    """
    Analyze implied volatility term structure.
    """
    
    @staticmethod
    def generate_term_structure(expirations, regime='normal'):
        """
        Generate term structure for different market regimes.
        
        Parameters:
        -----------
        expirations : array - Times to expiry in years
        regime : str - 'normal', 'inverted', or 'flat'
        
        Returns:
        --------
        array - ATM implied volatilities
        """
        if regime == 'normal':
            # Contango - near term lower than long term
            base_iv = 0.15
            term_effect = 0.08 * (1 - np.exp(-2 * expirations))
            noise = np.random.normal(0, 0.005, len(expirations))
            
        elif regime == 'inverted':
            # Backwardation - near term higher (stress)
            base_iv = 0.35
            term_effect = -0.15 * (1 - np.exp(-3 * expirations))
            noise = np.random.normal(0, 0.01, len(expirations))
            
        else:  # flat
            base_iv = 0.20
            term_effect = np.zeros_like(expirations)
            noise = np.random.normal(0, 0.005, len(expirations))
        
        return base_iv + term_effect + noise
    
    @staticmethod
    def calculate_forward_variance(ivs, expirations):
        """
        Calculate forward variance from term structure.
        
        Forward variance between T1 and T2:
        σ²(T1,T2) = (σ²(T2)*T2 - σ²(T1)*T1) / (T2 - T1)
        """
        total_variance = ivs**2 * expirations
        
        forward_var = []
        for i in range(len(expirations) - 1):
            fwd_var = (total_variance[i+1] - total_variance[i]) / (expirations[i+1] - expirations[i])
            forward_var.append(max(fwd_var, 0))  # Ensure non-negative
        
        forward_vol = np.sqrt(forward_var)
        return forward_vol
    
    @staticmethod
    def term_structure_metrics(ivs, expirations):
        """
        Calculate term structure metrics.
        """
        # Slope (linear regression)
        slope = np.polyfit(expirations, ivs, 1)[0]
        
        # Curvature (second derivative approximation)
        if len(ivs) >= 3:
            curvature = np.mean(np.diff(np.diff(ivs)))
        else:
            curvature = 0
        
        # Short/Long ratio
        short_iv = np.mean(ivs[:len(ivs)//3])
        long_iv = np.mean(ivs[-len(ivs)//3:])
        ratio = short_iv / long_iv if long_iv > 0 else np.nan
        
        return {
            'slope': slope,
            'curvature': curvature,
            'short_long_ratio': ratio,
            'regime': 'inverted' if ratio > 1 else 'normal'
        }


# Generate term structures for different regimes
expirations = np.array([7, 14, 21, 30, 45, 60, 90, 120, 180, 270, 365]) / 365

ts_analyzer = TermStructureAnalysis()

iv_normal = ts_analyzer.generate_term_structure(expirations, 'normal')
iv_inverted = ts_analyzer.generate_term_structure(expirations, 'inverted')
iv_flat = ts_analyzer.generate_term_structure(expirations, 'flat')

# Calculate forward volatility
fwd_vol_normal = ts_analyzer.calculate_forward_variance(iv_normal, expirations)
fwd_vol_inverted = ts_analyzer.calculate_forward_variance(iv_inverted, expirations)

# Plot term structures
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Spot IV Term Structure
ax1 = axes[0, 0]
ax1.plot(expirations * 365, iv_normal * 100, 'g-o', label='Normal (Contango)', linewidth=2, markersize=6)
ax1.plot(expirations * 365, iv_inverted * 100, 'r-s', label='Inverted (Backwardation)', linewidth=2, markersize=6)
ax1.plot(expirations * 365, iv_flat * 100, 'b-^', label='Flat', linewidth=2, markersize=6)
ax1.set_xlabel('Days to Expiry')
ax1.set_ylabel('Implied Volatility (%)')
ax1.set_title('ATM Term Structure - Different Regimes')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Forward Volatility
ax2 = axes[0, 1]
mid_exp = (expirations[:-1] + expirations[1:]) / 2
ax2.plot(mid_exp * 365, fwd_vol_normal * 100, 'g-o', label='Normal', linewidth=2, markersize=6)
ax2.plot(mid_exp * 365, fwd_vol_inverted * 100, 'r-s', label='Inverted', linewidth=2, markersize=6)
ax2.set_xlabel('Mid-point Days to Expiry')
ax2.set_ylabel('Forward Volatility (%)')
ax2.set_title('Forward Volatility Term Structure')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Total Variance
ax3 = axes[1, 0]
ax3.plot(expirations * 365, (iv_normal**2 * expirations) * 100, 'g-o', label='Normal', linewidth=2)
ax3.plot(expirations * 365, (iv_inverted**2 * expirations) * 100, 'r-s', label='Inverted', linewidth=2)
ax3.set_xlabel('Days to Expiry')
ax3.set_ylabel('Total Variance × 100')
ax3.set_title('Total Variance Curve')
ax3.legend()
ax3.grid(True, alpha=0.3)

# VIX-like index calculation
ax4 = axes[1, 1]

# Simulate VIX term structure (VIX, VIX3M, VIX6M, VIX1Y)
vix_tenors = ['VIX\n(30d)', 'VIX3M\n(90d)', 'VIX6M\n(180d)', 'VIX1Y\n(365d)']
vix_normal = [iv_normal[3], iv_normal[6], iv_normal[8], iv_normal[10]] * 100
vix_inverted = [iv_inverted[3], iv_inverted[6], iv_inverted[8], iv_inverted[10]] * 100

x = np.arange(len(vix_tenors))
width = 0.35
ax4.bar(x - width/2, vix_normal, width, label='Normal Market', color='green', alpha=0.7)
ax4.bar(x + width/2, vix_inverted, width, label='Stressed Market', color='red', alpha=0.7)
ax4.set_xticks(x)
ax4.set_xticklabels(vix_tenors)
ax4.set_ylabel('Implied Volatility (%)')
ax4.set_title('VIX Term Structure Comparison')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print metrics
print("\nTerm Structure Metrics:")
print("=" * 50)
for name, ivs in [('Normal', iv_normal), ('Inverted', iv_inverted), ('Flat', iv_flat)]:
    metrics = ts_analyzer.term_structure_metrics(ivs, expirations)
    print(f"\n{name} Regime:")
    print(f"  Slope:            {metrics['slope']*100:.4f}")
    print(f"  Short/Long Ratio: {metrics['short_long_ratio']:.4f}")
    print(f"  Detected Regime:  {metrics['regime']}")

---
## 6. Practical Application: Building an IV Engine

Let's build a complete implied volatility calculation engine that can:
1. Calculate IV for single options
2. Build volatility smiles
3. Construct full surfaces
4. Interpolate for arbitrary strikes/expirations

In [ ]:
class IVEngine:
    """
    Complete Implied Volatility calculation engine.
    """
    
    def __init__(self, S, r):
        """
        Initialize IV Engine.
        
        Parameters:
        -----------
        S : float - Current underlying price
        r : float - Risk-free rate
        """
        self.S = S
        self.r = r
        self.option_chain = []
        self.surface = None
    
    def add_option(self, K, T, price, option_type='call'):
        """
        Add an option to the chain and calculate its IV.
        """
        iv = ImpliedVolatility.brent(price, self.S, K, T, self.r, option_type)
        
        self.option_chain.append({
            'strike': K,
            'expiry': T,
            'type': option_type,
            'price': price,
            'iv': iv,
            'moneyness': np.log(K / self.S),
            'delta': self._calculate_delta(K, T, iv, option_type)
        })
        
        return iv
    
    def _calculate_delta(self, K, T, sigma, option_type):
        """Calculate option delta."""
        if np.isnan(sigma) or T <= 0:
            return np.nan
        d1 = BlackScholes.d1(self.S, K, T, self.r, sigma)
        if option_type == 'call':
            return norm.cdf(d1)
        else:
            return norm.cdf(d1) - 1
    
    def build_smile(self, T):
        """
        Extract volatility smile for a specific expiration.
        """
        smile_data = [opt for opt in self.option_chain if opt['expiry'] == T]
        smile_data.sort(key=lambda x: x['strike'])
        return smile_data
    
    def build_surface_from_chain(self):
        """
        Build interpolated volatility surface from option chain.
        """
        if not self.option_chain:
            raise ValueError("No options in chain")
        
        strikes = np.array([opt['strike'] for opt in self.option_chain])
        expiries = np.array([opt['expiry'] for opt in self.option_chain])
        ivs = np.array([opt['iv'] for opt in self.option_chain])
        
        # Remove NaN values
        valid = ~np.isnan(ivs)
        strikes = strikes[valid]
        expiries = expiries[valid]
        ivs = ivs[valid]
        
        self.surface = {
            'strikes': strikes,
            'expiries': expiries,
            'ivs': ivs
        }
        
        return self.surface
    
    def interpolate_iv(self, K, T, method='cubic'):
        """
        Interpolate IV for arbitrary strike and expiration.
        """
        if self.surface is None:
            self.build_surface_from_chain()
        
        points = np.column_stack([self.surface['strikes'], self.surface['expiries']])
        iv = griddata(points, self.surface['ivs'], (K, T), method=method)
        
        return float(iv)
    
    def get_summary(self):
        """
        Get summary statistics of the option chain.
        """
        if not self.option_chain:
            return None
        
        ivs = [opt['iv'] for opt in self.option_chain if not np.isnan(opt['iv'])]
        
        return {
            'num_options': len(self.option_chain),
            'mean_iv': np.mean(ivs),
            'median_iv': np.median(ivs),
            'min_iv': np.min(ivs),
            'max_iv': np.max(ivs),
            'iv_range': np.max(ivs) - np.min(ivs),
            'unique_strikes': len(set(opt['strike'] for opt in self.option_chain)),
            'unique_expiries': len(set(opt['expiry'] for opt in self.option_chain))
        }


# Demo: Build IV engine with synthetic data
S = 100
r = 0.05

engine = IVEngine(S, r)

# Generate synthetic option chain
strikes = np.linspace(80, 120, 9)
expirations = [30/365, 60/365, 90/365, 180/365]

print("Building Option Chain with Implied Volatility")
print("=" * 70)
print(f"{'Strike':<10} {'Expiry':<12} {'Price':<12} {'IV':<12} {'Delta'}")
print("-" * 70)

for T in expirations:
    for K in strikes:
        # Generate realistic IV with skew
        true_iv = 0.20 - 0.12 * np.log(K/S) + 0.01 * np.log(K/S)**2 - 0.03 * np.log(T/0.25)
        true_iv = max(true_iv, 0.08)
        
        # Calculate option price
        call_price = BlackScholes.call_price(S, K, T, r, true_iv)
        
        # Add to engine
        engine.add_option(K, T, call_price, 'call')

# Display sample
for opt in engine.option_chain[::4]:  # Every 4th option
    print(f"{opt['strike']:<10.1f} {opt['expiry']*365:<12.0f}d ${opt['price']:<11.2f} "
          f"{opt['iv']*100:<11.2f}% {opt['delta']:.3f}")

# Summary
print("\n" + "=" * 70)
summary = engine.get_summary()
print(f"Total Options: {summary['num_options']}")
print(f"IV Range: {summary['min_iv']*100:.2f}% - {summary['max_iv']*100:.2f}%")
print(f"Mean IV: {summary['mean_iv']*100:.2f}%")

In [ ]:
# Visualize the engine's volatility surface
engine.build_surface_from_chain()

# Create grid for interpolation
K_grid = np.linspace(80, 120, 41)
T_grid = np.linspace(0.05, 0.55, 41)
K_mesh, T_mesh = np.meshgrid(K_grid, T_grid)

# Interpolate
IV_mesh = np.zeros_like(K_mesh)
for i in range(len(T_grid)):
    for j in range(len(K_grid)):
        try:
            IV_mesh[i, j] = engine.interpolate_iv(K_grid[j], T_grid[i])
        except:
            IV_mesh[i, j] = np.nan

# Plot
fig = plt.figure(figsize=(14, 5))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax1.plot_surface(K_mesh, T_mesh * 365, IV_mesh * 100,
                        cmap='RdYlGn_r', alpha=0.9, edgecolor='none')
ax1.scatter([opt['strike'] for opt in engine.option_chain],
            [opt['expiry']*365 for opt in engine.option_chain],
            [opt['iv']*100 for opt in engine.option_chain],
            c='blue', s=20, alpha=0.8, label='Market Data')
ax1.set_xlabel('Strike')
ax1.set_ylabel('Days to Expiry')
ax1.set_zlabel('IV (%)')
ax1.set_title('Interpolated Volatility Surface')
ax1.view_init(elev=20, azim=45)

ax2 = fig.add_subplot(1, 2, 2)
contour = ax2.contourf(K_mesh, T_mesh * 365, IV_mesh * 100, levels=20, cmap='RdYlGn_r')
ax2.scatter([opt['strike'] for opt in engine.option_chain],
            [opt['expiry']*365 for opt in engine.option_chain],
            c='blue', s=30, alpha=0.8, label='Market Data')
ax2.axvline(x=S, color='black', linestyle='--', alpha=0.7, label='ATM')
plt.colorbar(contour, ax=ax2, label='IV (%)')
ax2.set_xlabel('Strike')
ax2.set_ylabel('Days to Expiry')
ax2.set_title('IV Surface Contour with Market Data Points')
ax2.legend()

plt.tight_layout()
plt.show()

---
## 7. Key Takeaways & Exercises

### Key Concepts Learned:
1. **Implied Volatility** is the market's expectation of future volatility implied by option prices
2. **Numerical methods** (Newton-Raphson, Bisection, Brent) are required to solve for IV
3. **Volatility smile/skew** shows IV varies with strike price
4. **Volatility surface** provides a complete view across strikes and expirations
5. **Term structure** reveals how IV changes with time to expiration

### Trading Applications:
- **Relative value**: Trade options that appear mispriced on the surface
- **Calendar spreads**: Exploit term structure anomalies
- **Skew trading**: Trade the difference between OTM puts and calls
- **Volatility forecasting**: Use IV as forward-looking volatility estimate

In [ ]:
# Exercise 1: IV Sensitivity Analysis
print("EXERCISE 1: IV Sensitivity to Option Price")
print("=" * 50)

# How much does IV change when option price changes by 1%?
S, K, T, r = 100, 100, 0.25, 0.05
base_iv = 0.20
base_price = BlackScholes.call_price(S, K, T, r, base_iv)

print(f"Base Price: ${base_price:.4f}")
print(f"Base IV: {base_iv*100:.2f}%")
print("\nPrice Change → IV Change:")

for pct_change in [-5, -2, -1, 1, 2, 5]:
    new_price = base_price * (1 + pct_change/100)
    new_iv = ImpliedVolatility.brent(new_price, S, K, T, r)
    iv_change = (new_iv - base_iv) * 100
    print(f"  Price {pct_change:+.0f}%: IV → {new_iv*100:.2f}% ({iv_change:+.2f}% pts)")

In [ ]:
# Exercise 2: Detect Arbitrage Opportunities
print("\nEXERCISE 2: Calendar Spread Arbitrage Detection")
print("=" * 50)

def check_calendar_arbitrage(iv_near, iv_far, T_near, T_far):
    """
    Check if term structure violates no-arbitrage conditions.
    Total variance must be increasing in time.
    """
    var_near = iv_near**2 * T_near
    var_far = iv_far**2 * T_far
    
    return var_far < var_near  # Arbitrage if far variance < near variance


# Test cases
test_cases = [
    {'iv_near': 0.30, 'iv_far': 0.20, 'T_near': 0.08, 'T_far': 0.25},  # Potential arbitrage
    {'iv_near': 0.20, 'iv_far': 0.22, 'T_near': 0.08, 'T_far': 0.25},  # Normal
    {'iv_near': 0.35, 'iv_far': 0.18, 'T_near': 0.08, 'T_far': 0.25},  # Arbitrage
]

for i, tc in enumerate(test_cases, 1):
    has_arb = check_calendar_arbitrage(**tc)
    var_near = tc['iv_near']**2 * tc['T_near']
    var_far = tc['iv_far']**2 * tc['T_far']
    
    print(f"\nCase {i}:")
    print(f"  Near: IV={tc['iv_near']*100:.0f}%, T={tc['T_near']*365:.0f}d, Var={var_near:.4f}")
    print(f"  Far:  IV={tc['iv_far']*100:.0f}%, T={tc['T_far']*365:.0f}d, Var={var_far:.4f}")
    print(f"  Arbitrage: {'⚠️ YES' if has_arb else '✓ NO'}")

In [ ]:
# Summary statistics
print("\n" + "=" * 60)
print("NOTEBOOK SUMMARY: Implied Volatility")
print("=" * 60)
print("""
✓ Implemented Black-Scholes pricing with Greeks
✓ Built IV solvers: Newton-Raphson, Bisection, Brent
✓ Analyzed volatility smile and skew patterns
✓ Constructed 3D volatility surfaces
✓ Explored term structure dynamics
✓ Created a complete IV calculation engine

Key Formulas:
• Black-Scholes: C = S*N(d1) - K*e^(-rT)*N(d2)
• Newton-Raphson: σ(n+1) = σ(n) - (C_BS - C_mkt) / Vega
• Forward Variance: σ²(T1,T2) = (σ²T2*T2 - σ²T1*T1)/(T2-T1)

Next Steps:
→ Day 3: Options Greeks and Hedging
→ Day 4: Deep Hedging with Neural Networks
""")